# Clash Royale Deck Neural Netwroks Model

## Initialize

In [ ]:
#general
import pandas as pd
import numpy as np

# For csv merging from folder
import os
# For parsing key-value pairs in dataframe
import ast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Load data

In [ ]:
# Set the path to the directory containing the CSV files
path = '/content/drive/MyDrive/Hackathon 2023/Data'

# Create an empty list to store the DataFrames
dfs = []

# Iterate through all files in the directory and check if the file is a CSV file
for file in os.listdir(path):
    if file.endswith('.csv'):
        # read the CSV file into a DataFrame and append it to the list
        df = pd.read_csv(os.path.join(path, file))
        dfs.append(df)

# Concatenate all DataFrames into one final DataFrame
dfCls = pd.concat(dfs, axis=0, ignore_index=True)

In [ ]:
# Drop empty data
dfCls = dfCls.dropna()

In [ ]:
pd.set_option('display.max_colwidth', 50)
dfCls.head()

## Quick Analysis

### Histogram of Trophy Levels

In [ ]:
import matplotlib.pyplot as plt

# Create a histogram with 10 bins
plt.hist(dfCls['startingTrophies2'],bins = 20)

# Add x and y labels and a title
plt.xlabel('Trophy Level of Players')
plt.ylabel('Frequency')
plt.title('Histogram of Trophy Levels of Players')

# Display the histogram
plt.show()

### Plot of Game Types

In [ ]:
dfCls['type'].describe()

In [ ]:
import matplotlib.pyplot as plt

# Create a histogram with 10 bins
plt.hist(dfCls['type'])

# Add x and y labels and a title
plt.xlabel('Types of Game',)
plt.ylabel('Frequency')
plt.title('Histogram of Game Types')
plt.xticks(fontsize = 3)

# Display the histogram
plt.show()

### Filter type to PvP

In [ ]:
# Filter to PvP
unique_values = ['PvP']
dfCls = dfCls[dfCls['type'].isin(unique_values)]
dfCls.describe()
dfCls.head()
dfCls['type'].describe()

### Rerun Histogram of Trophies

In [ ]:
import matplotlib.pyplot as plt

# Create a histogram with 10 bins
plt.hist(dfCls['startingTrophies2'],bins = 20)

# Add x and y labels and a title
plt.xlabel('Trophy Level of Players')
plt.ylabel('Frequency')
plt.title('Histogram of Trophy Levels of Players')

# Display the histogram
plt.show()

## Take a subset for data analysis

In [ ]:
sample_battle_data_df = dfCls.sample(n=10000,random_state=1)

## Data Preprocessing

In [ ]:
# Parsing card data for player 1
sample_battle_data_df['cards1_dict'] = sample_battle_data_df['cards1'].apply(lambda x: ast.literal_eval(x))
sample_battle_data_df['deck1'] = sample_battle_data_df['cards1_dict'].apply(lambda x: [a['name'] for a in x])
sample_battle_data_df['max_level_1'] = sample_battle_data_df['cards1_dict'].apply(lambda x: [a['maxLevel'] for a in x])
sample_battle_data_df['level_1'] = sample_battle_data_df['cards1_dict'].apply(lambda x: [a['level'] for a in x])

# Parsing card data for player 2
sample_battle_data_df['cards2_dict'] = sample_battle_data_df['cards2'].apply(lambda x: ast.literal_eval(x))
sample_battle_data_df['deck2'] = sample_battle_data_df['cards2_dict'].apply(lambda x: [a['name'] for a in x])
sample_battle_data_df['max_level_2'] = sample_battle_data_df['cards2_dict'].apply(lambda x: [a['maxLevel'] for a in x])
sample_battle_data_df['level_2'] = sample_battle_data_df['cards2_dict'].apply(lambda x: [a['level'] for a in x])

In [ ]:
# Drop key-value pair columns now that data is extracted
sample_battle_data_df.drop(['cards1','cards2','cards1_dict','cards2_dict'],axis=1,inplace=True)

In [ ]:
# Function that merges two lists by making a list of tuple values of each list
def merge(list1, list2):
    merged_list = [(list1[i], list2[i]) for i in range(len(list1))]
    return merged_list

In [ ]:
# Because Clash Royale used to have a very poorly designed leveling system, and it's analytics hasn't changed, it's analytics still tracks it using the old system
# Using formula below from google to scale it to new level system

sample_battle_data_df['cards_level_1'] = [[x-y+14 for x,y in merge(sample_battle_data_df['level_1'].iloc[i],sample_battle_data_df['max_level_1'].iloc[i])] 
                                          for i in range(len(sample_battle_data_df))]
sample_battle_data_df['cards_level_2'] = [[x-y+14 for x,y in merge(sample_battle_data_df['level_2'].iloc[i],sample_battle_data_df['max_level_2'].iloc[i])] 
                                          for i in range(len(sample_battle_data_df))]

In [ ]:
# Averaging card levels from each player's deck
sample_battle_data_df['avg_cards_level_1'] = [sum(x)/8 for x in sample_battle_data_df['cards_level_1']]
sample_battle_data_df['avg_cards_level_2'] = [sum(x)/8 for x in sample_battle_data_df['cards_level_2']]

# Calculate the card level differential (Player 2 Avg - Player 1 Avg)
sample_battle_data_df['avg_cards_level_diff'] = sample_battle_data_df['avg_cards_level_2']-sample_battle_data_df['avg_cards_level_1']

In [ ]:
# Calculate level differential for player one
sample_battle_data_df['level_differential'] = sample_battle_data_df['avg_cards_level_1'] - sample_battle_data_df['avg_cards_level_2']

In [ ]:
# calculate winner
sample_battle_data_df['crowns_won'] = sample_battle_data_df['crowns2'] - sample_battle_data_df['crowns1']
sample_battle_data_df['winner'] = 0
sample_battle_data_df['winner'].loc[sample_battle_data_df['crowns_won']>0] = 2
sample_battle_data_df['winner'].loc[sample_battle_data_df['crowns_won']<0] = 1

# Create binary winner value for player one
sample_battle_data_df['player_one_is_winner'] = sample_battle_data_df['winner']
sample_battle_data_df['player_one_is_winner'] = sample_battle_data_df['player_one_is_winner'].replace(2,0)
sample_battle_data_df.head()
                              

## Create a new dataframe of deck related variables for player one

In [ ]:
# New dataframe for neural networks model
dfnn = sample_battle_data_df.loc[:,['deck1','level_differential','player_one_is_winner']]
dfnn.head()

## Read in card csv

In [ ]:
dfCds = pd.read_csv('/content/drive/MyDrive/Hackathon 2023/Card Stats/card_info.csv')
dfCds.head()

## Create dataframe of cards x deck matrix

In [ ]:
# create a dataframe for the decks
df = pd.DataFrame(dfnn['deck1'])

# create a list of all the card names
all_names = dfCds['name']

# create a new dataframe with a column for each name
new_df = pd.DataFrame(columns=all_names)

# loop through each combination of names
for i, row in df.iterrows():
    combination = row['deck1']
    name_values = []
    # loop through each name and check if it's in the combination
    for name in all_names:
        if name in combination:
            name_values.append(1)
        else:
            name_values.append(0)
    # add the name values as a new row in the new dataframe
    new_df.loc[i] = name_values

In [ ]:
# Recreate the neural networks dataframe
dfnn = new_df

## Scale data

In [ ]:
from sklearn.preprocessing import scale

#dfnn['level_differential'] = scale(dfnn['level_differential'])
dfnn.head()

In [ ]:
dfnn['winner'] = sample_battle_data_df['player_one_is_winner']

# Build Neural Network Model

In [ ]:
from sklearn.model_selection import train_test_split

y = dfnn.loc[:,'winner']
X = dfnn.drop(['winner'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
from sklearn.neural_network import MLPClassifier

mdlNncDgt = MLPClassifier()
mdlNncDgt.fit(X_train, y_train)
y_test_pred = mdlNncDgt.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_test_pred))


# Scatterplot of level differential vs winner

In [ ]:
# Scatter plot

plt.scatter(x=dfnn['winner'], y=dfnn['level_differential'])